In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx
import optuna
import re

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer



from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('TARGET')
modules = Modules('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')

In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [4]:
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 2000
}

In [5]:
apps, prev, bureau, bureau_bal, pos_bal, install, card_bal = FeatureEngineering3.get_dataset()
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df = FeatureEngineering3.get_apps_all_with_all_agg(apps, prev, bureau, bureau_bal, pos_bal, install, card_bal)
df.head()

prev_agg shape: (338857, 41) bureau_agg shape: (305811, 103)
pos_bal_agg shape: (337252, 45) install_agg shape: (339587, 59) card_bal_agg shape: (103558, 70)
apps_all before merge shape: (356255, 152)
apps_all after merge with all shape: (356255, 466)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [8]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [10]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [11]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 603 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 593 features are remained after removing non-informative features
60 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 533 features are remained after removing features not interesting for LightGBM classifier
---=> 38 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 571 features

---=> df final shape: (356250, 571)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,1,0,0,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,1,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,1,0,0,0,0,0,0,0,0


In [12]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 571 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 567 features are remained after removing non-informative features
22 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 545 features are remained after removing features not interesting for LightGBM classifier
---=> 15 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 560 features

---=> df final shape: (356250, 560)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [14]:
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,1,1,0,0,0,0,0,0
307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,1,1,0,0,0,0,0,0
307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,0,1,0,1,0,1,0,1,0
307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,0,1,1,1,0,1,0,1,0
307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.042863
1,100005,0.140563
2,100013,0.025033
3,100028,0.032262
4,100038,0.205365


In [16]:
df_train = df_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,1,1,0,0,0,0,0,0,0.042863
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,1,1,0,0,0,0,0,0,0.140563
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,1,0,0.025033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,1,1,0,1,0,1,0,0.032262
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0.205365


In [17]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.042863
1,100005,0.140563
2,100013,0.025033
3,100028,0.032262
4,100038,0.205365


In [18]:
df_train = df_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,1,0,0,0,0,0,0,0.042863,0.038227
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,1,0,0,0,0,0,0,0.140563,0.112768
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,1,0,0.025033,0.033638
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,1,0,1,0,1,0,0.032262,0.031213
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0.205365,0.181109


In [19]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [20]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,0,0,0,0,0,0,0.042863,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,0,0,0,0,0,0,0.140563,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,0.025033,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,0,1,0,1,0,0.032262,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0.205365,0.181109,0.172674


In [21]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [22]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0,0,0,0.042863,0.038227,0.025023,0.040068
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0,0,0,0.140563,0.112768,0.090335,0.095240
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,0.025033,0.033638,0.037033,0.044435
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,0,1,0,1,0,0.032262,0.031213,0.036644,0.036586
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0.205365,0.181109,0.172674,0.141754


In [23]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,0,0,0,0,0.284542,0.162368,0.156048,0.260928
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,0,0,0,0,0.007952,0.027249,0.012034,0.012171
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0.013653,0.030869,0.026333,0.039950
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,1,0,1,0,1,0,0.033349,0.029480,0.029856,0.030776
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,0,0,0,0,0.071830,0.047569,0.034698,0.035088


In [24]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0,0,0,0.042863,0.038227,0.025023,0.040068
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0,0,0,0.140563,0.112768,0.090335,0.095240
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,0.025033,0.033638,0.037033,0.044435
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,0,1,0,1,0,0.032262,0.031213,0.036644,0.036586
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0.205365,0.181109,0.172674,0.141754


In [25]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [26]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0,0,0,0,0,0,0.284542,0.162368,0.156048,0.260928
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,0,0,0,0,0,0,0.007952,0.027249,0.012034,0.012171
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0,0,0,0,0,0,0.013653,0.030869,0.026333,0.039950
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,1,0,1,0,1,0,0.033349,0.029480,0.029856,0.030776
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0,0,0,0,0,0,0.071830,0.047569,0.034698,0.035088


In [27]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,2.325571,0.986709,0.860543,1.896448
100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.810405,-0.548398,-0.659948,-0.711865
100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.745770,-0.507271,-0.508977,-0.420592
100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.522451,-0.523054,-0.471789,-0.516790
100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.086162,-0.317542,-0.420660,-0.471570


In [28]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,2.325571,0.986709,0.860543,1.896448
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.810405,-0.548398,-0.659948,-0.711865
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.745770,-0.507271,-0.508977,-0.420592
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.522451,-0.523054,-0.471789,-0.516790
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.086162,-0.317542,-0.420660,-0.471570


In [29]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 697.96 MB
Memory usage of optimization is 693733664.00 MB
Decreased by -99394331.9%
Memory usage of dataframe is 110.54 MB
Memory usage of optimization is 109771616.00 MB
Decreased by -99301552.6%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a,set1
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,2.326172,0.986816,0.860352,1.896484
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.810547,-0.548340,-0.660156,-0.711914
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.745605,-0.507324,-0.508789,-0.420654
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,1.693359,-0.138672,1.614258,0.0,1.614258,0.0,-0.522461,-0.522949,-0.471680,-0.516602
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.086182,-0.317627,-0.420654,-0.471680


In [30]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [31]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [32]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 562) (61502, 562)
[100]	training's auc: 0.798498	valid_1's auc: 0.787307
[200]	training's auc: 0.803751	valid_1's auc: 0.789658
[300]	training's auc: 0.807626	valid_1's auc: 0.790982
[400]	training's auc: 0.811372	valid_1's auc: 0.792356
[500]	training's auc: 0.814911	valid_1's auc: 0.793116
[600]	training's auc: 0.818357	valid_1's auc: 0.79355
[700]	training's auc: 0.821577	valid_1's auc: 0.793937
[800]	training's auc: 0.824616	valid_1's auc: 0.794112
[900]	training's auc: 0.827554	valid_1's auc: 0.794346
[1000]	training's auc: 0.830394	valid_1's auc: 0.794493
[1100]	training's auc: 0.833091	valid_1's auc: 0.794639
[1200]	training's auc: 0.835819	valid_1's auc: 0.794619
[1300]	training's auc: 0.838406	valid_1's auc: 0.794587
[1400]	training's auc: 0.84088	valid_1's auc: 0.794661
[1500]	training's auc: 0.843265	valid_1's auc: 0.794599
[1600]	training's auc: 0.845732	valid_1's auc: 0.794525
[1700]	training's auc: 0.848005	valid_1's au

In [33]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
21,APPS_ANNUITY_CREDIT_RATIO,1186.2,70.892172
561,set2a,739.0,26.692696
38,APP_EXT_SOURCE_2_EXT_SOURCE_3,632.8,28.542950
253,EXT_SOURCE_3,595.2,44.024993
0,AMT_ANNUITY,589.0,27.449954
49,APP_SCORE2_TO_POP_RATIO,575.6,22.434349
233,DAYS_EMPLOYED,569.0,28.696690
145,BUREAU_DAYS_CREDIT_MAX,556.2,19.816660
235,DAYS_LAST_PHONE_CHANGE,539.8,29.482198
559,pred_1b,523.4,18.968395


In [34]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [35]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [36]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_OS_3-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.040141
1,100005,0.098777
2,100013,0.061508
3,100028,0.039286
4,100038,0.162628


## パラメータ・チューニング

In [37]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [38]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [39]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [40]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [41]:
### 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_OS_3-2.csv", index=None)